In [ ]:
import os
import pandas as pd
import re
import math
from datetime import datetime

# 抓取數據的資料夾路徑
FilePath = '/py_env_data/24024_14100/'
allFileList = os.listdir(FilePath)
filelist = [file for file in allFileList if os.path.isfile(os.path.join(FilePath, file)) and file.endswith('.csv')]

def parse_time(time_str):
    # 拆解資料中時間格式 --> 2024y1m11d_pm 11:00:21.486
    try:
        date_part, time_part = time_str.split('_')
        year_part, month_day_part = date_part.split('y')
        year = int(year_part)
        month_day = month_day_part.split('m')
        month = int(month_day[0])
        day = int(month_day[1].split('d')[0])
        time_parts = time_part.split(' ')
        period = time_parts[0]
        hour_minute_second = time_parts[1].split(':')
        hour = int(hour_minute_second[0])
        minute = int(hour_minute_second[1])
        second_parts = hour_minute_second[2].split('.')
        second = int(second_parts[0])
        microsecond = int(second_parts[1]) * 1000  # 將毫秒轉換為微秒

        if period == 'pm' and hour < 12:
            hour += 12
        elif period == 'am' and hour == 12:
            hour = 0

        dt = datetime(year, month, day, hour, minute, second, microsecond)
        return dt.timestamp()  # 返回時間戳的數值
    except Exception as e:
        print(f"Error parsing time: {time_str}, {e}")
        return None

def process_file(file_path):
    try:
        df = pd.read_csv(file_path)  # 資料匯入DataFrame
        df['Time'] = df['Time'].apply(parse_time)
        df['X'] = df['Time'].diff()
        df.iloc[0, 0] = 0
        start_idx = 1 #刪除欄位起點
        end_idx = 16 #刪除欄位終點
        df = df.drop(df.columns[start_idx:end_idx], axis=1) #刪除不需要的欄位
        return df
    except Exception as e:
        print(f"Error processing file: {file_path}, {e}")
        return None

def calculate_values(df):
    # 假設Pin, PM, EFF的計算公式如下，根據實際情況修改
    df['Pin'] = math.sqrt(3) * df['Urms_AVG_A'] * df['Irms_AVG_A'] * abs(df['PF_AVG_A']) / 1000  # unit: kW
    df['PM'] = 2 * math.pi * (df['FREQ_W_A'] * 120 / 8) * df['Torque_A'] / 60000  # unit: kW
    df['EFF'] = df['PM'] / df['Pin'] * 100  # 假設EFF為PM與Pin的比值
    # print(df)
    return df

def save_to_excel(df, df_name, func_match, Avg_point):
    try:
        # 計算Pin, PM, EFF並添加到DataFrame中
        df = calculate_values(df)
        sort_AVG = df.tail(Avg_point).mean()
        sort_MAX = df.max()
        sort_min = df.min()

        # 新增Pin, PM, EFF的計算值
        Pin_avg = df['Pin'].mean()
        PM_avg = df['PM'].mean()
        EFF_avg = df['EFF'].mean()

        # 將這些值添加到結果中
        sort_results = pd.concat([pd.Series({'Pin': Pin_avg, 'PM': PM_avg, 'EFF': EFF_avg}), sort_AVG, sort_MAX, sort_min], axis=1, keys=["Calculated", "AVG.", "MAX", "Min"])

        #篩選出的平均數據在根目錄
        # excel_file = f"{df_name}_results.xlsx"

        new_folder = func_match.group()
        current_dir = os.getcwd()
        new_path = os.path.join(current_dir, new_folder)
        if not os.path.exists(new_path):
            os.makedirs(new_path)

        # 更新excel_file和LastRAW_excel_file路径
        excel_file = os.path.join(new_path, f"{df_name}_results.xlsx")
        LastRAW_excel_file = os.path.join(new_path, f'Last{Avg_point}_{df_name}.xlsx')

        sheet_name_sortout = df_name

        #存儲平均後數據
        if os.path.isfile(excel_file):
            path = os.path.join(os.getcwd(), excel_file)
            with pd.ExcelWriter(engine='openpyxl', path=path, mode='a', if_sheet_exists='replace') as writer:
                sort_results.to_excel(writer, sheet_name=sheet_name_sortout)
        else:
            with pd.ExcelWriter(excel_file) as writer:
                sort_results.to_excel(writer, sheet_name=sheet_name_sortout)

        new_folder = func_match.group()
        current_dir = os.getcwd()
        new_path = os.path.join(current_dir, new_folder)
        if not os.path.exists(new_path):
            os.makedirs(new_path)

        #存儲原始數據資料
        LastRAW_excel_file = os.path.join(new_folder, f'Last{Avg_point}_{df_name}.xlsx')
        sheet_name_RAW = df_name
        print(df.tail(Avg_point))

        if os.path.isfile(LastRAW_excel_file):
            path = os.path.join(new_path, LastRAW_excel_file)
            with pd.ExcelWriter(engine='openpyxl', path=path, mode='a', if_sheet_exists='replace') as writer:
                df.tail(Avg_point).to_excel(writer, sheet_name=sheet_name_RAW)
        else:
            with pd.ExcelWriter(LastRAW_excel_file) as writer:
                df.tail(Avg_point).to_excel(writer, sheet_name=sheet_name_RAW)
    except Exception as e:
        print(f"Error saving to excel: {df_name}, {e}")

# 使用迴圈依次處理每個 .csv 檔案建立DataFrame
all_dfs = {}
for i, file in enumerate(filelist, start=1):
    file_path = os.path.join(FilePath, file)
    df = process_file(file_path)
    if df is not None:
        file_name, _ = os.path.splitext(file)
        df_name = file_name
        all_dfs[df_name] = df #將df放入all_dfs[檔案名稱]成為新的陣列
        # print(all_dfs)

# 迴圈處理所有 DataFrame
Tolerance_const = 0.988
for df_name, df in all_dfs.items():
    if isinstance(df, pd.DataFrame):
        Time_inver = df['X'].mean()
        name_parts = df_name.split('_')


        if len(name_parts) >= 4:
            func_match = re.search(r'\w+', name_parts[0])
            speed_match = re.search(r'\d+', name_parts[2])
            current_match = re.search(r'\d+', name_parts[3])

            # print('test conditon: ' + str(combined_condition.bool))
            print('test item: ' + str(func_match))
            print('Speed tar: ' + str(speed_match))
            print('Current tar: ' + str(current_match ))
            if func_match.group() == 'Continuous':
                tar_speed = int(speed_match.group()) * 8 / 120
                tar_current = int(current_match.group())
                Avg_point = 60
            elif func_match.group() == 'Peak':
                tar_speed = int(speed_match.group()) * 8 / 120
                tar_current = int(current_match.group())
                Avg_point = 2
            elif func_match.group() == 'SCC':
                tar_speed = 4000 * Tolerance_const
                tar_current = int(current_match.group())
                Avg_point = 5
            elif func_match.group() == 'BEMF':
                tar_speed = 1000 * Tolerance_const
                tar_current = int(current_match.group())
                Avg_point = 10
        else:

            print('Your length of file name does not match the program requirements!')
            continue

        #誤差值抓取0.05%
        if func_match.group() == 'SCC':
            condition_1 = abs(df['Speed_A']) >= tar_speed * 0.988
            condition_2 = df['Irms_AVG_A'] >= tar_current * Tolerance_const
        else:
            condition_1 = abs(df['FREQ_U_A']) >= tar_speed * Tolerance_const
            condition_2 = df['Irms_AVG_A'] >= tar_current * Tolerance_const

        #比較條件
        combined_condition = condition_1 & condition_2
        df = df.loc[combined_condition]
        df = df.astype(float)

        # 計算Pin, PM, EFF並添加到DataFrame中
        # df = calculate_values(df)

        save_to_excel(df, df_name, func_match, Avg_point)

In [2]:
#修改為處理完的資料放回父資料夾名+ "_results"
import os
import pandas as pd
import re
import math
from datetime import datetime

# 抓取數據的資料夾路徑
FilePath = '/py_env_data/24024_14100/'
ResultsPath = os.path.join(FilePath, 'results')  # 動態生成結果資料夾的路徑
if not os.path.exists(ResultsPath):
    os.makedirs(ResultsPath)  # 創建資料夾

allFileList = os.listdir(FilePath)
filelist = [file for file in allFileList if os.path.isfile(os.path.join(FilePath, file)) and file.endswith('.csv')]

def parse_time(time_str):
    # 拆解資料中時間格式 --> 2024y1m11d_pm 11:00:21.486
    try:
        date_part, time_part = time_str.split('_')
        year_part, month_day_part = date_part.split('y')
        year = int(year_part)
        month_day = month_day_part.split('m')
        month = int(month_day[0])
        day = int(month_day[1].split('d')[0])
        time_parts = time_part.split(' ')
        period = time_parts[0]
        hour_minute_second = time_parts[1].split(':')
        hour = int(hour_minute_second[0])
        minute = int(hour_minute_second[1])
        second_parts = hour_minute_second[2].split('.')
        second = int(second_parts[0])
        microsecond = int(second_parts[1]) * 1000  # 將毫秒轉換為微秒

        if period == 'pm' and hour < 12:
            hour += 12
        elif period == 'am' and hour == 12:
            hour = 0

        dt = datetime(year, month, day, hour, minute, second, microsecond)
        return dt.timestamp()  # 返回時間戳的數值
    except Exception as e:
        print(f"Error parsing time: {time_str}, {e}")
        return None

def process_file(file_path):
    try:
        df = pd.read_csv(file_path)  # 資料匯入DataFrame
        df['Time'] = df['Time'].apply(parse_time)
        df['X'] = df['Time'].diff()
        df.iloc[0, 0] = 0
        start_idx = 1 #刪除欄位起點
        end_idx = 16 #刪除欄位終點
        df = df.drop(df.columns[start_idx:end_idx], axis=1) #刪除不需要的欄位
        return df
    except Exception as e:
        print(f"Error processing file: {file_path}, {e}")
        return None

def calculate_values(df):
    # 假設Pin, PM, EFF的計算公式如下，根據實際情況修改
    df['Pin'] = math.sqrt(3) * df['Urms_AVG_A'] * df['Irms_AVG_A'] * abs(df['PF_AVG_A']) / 1000  # unit: kW
    df['PM'] = 2 * math.pi * (df['FREQ_W_A'] * 120 / 8) * df['Torque_A'] / 60000  # unit: kW
    df['EFF'] = df['PM'] / df['Pin'] * 100  # 假設EFF為PM與Pin的比值
    # print(df)
    return df

def save_to_excel(df, df_name, func_match, Avg_point, results_folder):
    try:
        # 計算Pin, PM, EFF並添加到DataFrame中
        df = calculate_values(df)
        sort_AVG = df.tail(Avg_point).mean()
        sort_MAX = df.max()
        sort_min = df.min()

        # 新增Pin, PM, EFF的計算值
        Pin_avg = df['Pin'].mean()
        PM_avg = df['PM'].mean()
        EFF_avg = df['EFF'].mean()

        # 將這些值添加到結果中
        sort_results = pd.concat([pd.Series({'Pin': Pin_avg, 'PM': PM_avg, 'EFF': EFF_avg}), sort_AVG, sort_MAX, sort_min], axis=1, keys=["Calculated", "AVG.", "MAX", "Min"])

        # 根據 func_match.group() 創建子資料夾
        func_folder = func_match.group()
        func_folder_path = os.path.join(results_folder, func_folder)
        if not os.path.exists(func_folder_path):
            os.makedirs(func_folder_path)  # 創建子資料夾

        # 更新excel_file和LastRAW_excel_file路径
        excel_file = os.path.join(func_folder_path, f"{df_name}_results.xlsx")  # 使用子資料夾路徑
        LastRAW_excel_file = os.path.join(func_folder_path, f'Last{Avg_point}_{df_name}.xlsx')  # 使用子資料夾路徑

        sheet_name_sortout = df_name

        # 存儲平均後數據
        if os.path.isfile(excel_file):
            with pd.ExcelWriter(engine='openpyxl', path=excel_file, mode='a', if_sheet_exists='replace') as writer:
                sort_results.to_excel(writer, sheet_name=sheet_name_sortout)
        else:
            with pd.ExcelWriter(excel_file) as writer:
                sort_results.to_excel(writer, sheet_name=sheet_name_sortout)

        # 存儲原始數據資料
        sheet_name_RAW = df_name
        print(df.tail(Avg_point))

        if os.path.isfile(LastRAW_excel_file):
            with pd.ExcelWriter(engine='openpyxl', path=LastRAW_excel_file, mode='a', if_sheet_exists='replace') as writer:
                df.tail(Avg_point).to_excel(writer, sheet_name=sheet_name_RAW)
        else:
            with pd.ExcelWriter(LastRAW_excel_file) as writer:
                df.tail(Avg_point).to_excel(writer, sheet_name=sheet_name_RAW)
    except Exception as e:
        print(f"Error saving to excel: {df_name}, {e}")

# 使用迴圈依次處理每個 .csv 檔案建立DataFrame
all_dfs = {}
for i, file in enumerate(filelist, start=1):
    file_path = os.path.join(FilePath, file)
    df = process_file(file_path)
    if df is not None:
        file_name, _ = os.path.splitext(file)
        df_name = file_name
        all_dfs[df_name] = df  # 將df放入all_dfs[檔案名稱]成為新的陣列

# 迴圈處理所有 DataFrame
Tolerance_const = 0.988
for df_name, df in all_dfs.items():
    if isinstance(df, pd.DataFrame):
        Time_inver = df['X'].mean()
        name_parts = df_name.split('_')

        if len(name_parts) >= 4:
            func_match = re.search(r'\w+', name_parts[0])
            speed_match = re.search(r'\d+', name_parts[2])
            current_match = re.search(r'\d+', name_parts[3])

            print('test item: ' + str(func_match))
            print('Speed tar: ' + str(speed_match))
            print('Current tar: ' + str(current_match))
            if func_match.group() == 'Continuous':
                tar_speed = int(speed_match.group()) * 8 / 120
                tar_current = int(current_match.group())
                Avg_point = 60
            elif func_match.group() == 'Peak':
                tar_speed = int(speed_match.group()) * 8 / 120
                tar_current = int(current_match.group())
                Avg_point = 2
            elif func_match.group() == 'SCC':
                tar_speed = 4000 * Tolerance_const
                tar_current = int(current_match.group())
                Avg_point = 5
            elif func_match.group() == 'BEMF':
                tar_speed = 1000 * Tolerance_const
                tar_current = int(current_match.group())
                Avg_point = 10
        else:
            print('Your length of file name does not match the program requirements!')
            continue

        # 誤差值抓取0.05%
        if func_match.group() == 'SCC':
            condition_1 = abs(df['Speed_A']) >= tar_speed * 0.988
            condition_2 = df['Irms_AVG_A'] >= tar_current * Tolerance_const
        else:
            condition_1 = abs(df['FREQ_U_A']) >= tar_speed * Tolerance_const
            condition_2 = df['Irms_AVG_A'] >= tar_current * Tolerance_const

        # 比較條件
        combined_condition = condition_1 & condition_2
        df = df.loc[combined_condition]
        df = df.astype(float)

        # 保存到 Excel
        save_to_excel(df, df_name, func_match, Avg_point, ResultsPath)  # 傳遞新的資料夾路徑


test item: <re.Match object; span=(0, 10), match='Continuous'>
Speed tar: <re.Match object; span=(0, 4), match='1000'>
Current tar: <re.Match object; span=(0, 3), match='200'>
          X  Urms_U_A  Urms_V_A  Urms_W_A  Urms_AVG_A  Irms_U_A  Irms_V_A  \
1294  1.260     83.55     83.37     83.49       83.47    200.01    199.88   
1295  0.273     83.55     83.37     83.49       83.47    200.03    199.87   
1296  1.248     83.55     83.37     83.49       83.47    200.02    199.89   
1297  2.315     83.56     83.38     83.50       83.48    200.01    199.89   
1298  1.306     83.54     83.37     83.49       83.47    199.99    199.90   
1299  0.311     83.54     83.36     83.48       83.46    200.00    199.90   
1300  2.284     83.54     83.37     83.48       83.46    199.99    199.92   
1301  1.294     83.55     83.38     83.50       83.48    200.00    199.92   
1302  0.237     83.56     83.38     83.51       83.48    200.00    199.89   
1303  1.279     83.55     83.37     83.50       83.47 